<a href="https://colab.research.google.com/github/Alfred78w/AI_project/blob/main/AI_generative_de_texte_comme_nietzshe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import keras
import numpy as np
import random
import sys
path = keras.utils.get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('Corpus length:', len(text))

Corpus length: 600893


You’ll extract sequences of 60 characters.
You’ll sample a new sequence every three characters.
Holds the extracted sequences
Holds the targets (the follow-up characters)

In [ ]:
maxlen = 60
step = 3
sentences = []
next_chars = []

Holds the targets (the follow-up characters).
List of unique characters in the corpus. Dictionary that maps unique characters to their index in the list “chars”


In [ ]:
for i in range(0, len(text) - maxlen, step):
  sentences.append(text[i: i + maxlen])
  next_chars.append(text[i + maxlen])
print('Number of sequences:', len(sentences))

chars = sorted(list(set(text)))
print('Unique characters:', len(chars))
char_indices = dict((char, chars.index(char)) for char in chars)

Number of sequences: 400556
Unique characters: 57


One-hot encodes
the characters
into binary arrays

In [ ]:
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool_)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool_)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [ ]:
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=bool)
y = np.zeros((len(sentences), len(chars)), dtype=bool)
for i, sentence in enumerate(sentences):
  for t, char in enumerate(sentence):
    x[i, t, char_indices[char]] = 1
  y[i, char_indices[next_chars[i]]] = 1

Vectorization...


 Single-layer LSTM model for next-character prediction

In [ ]:
from keras import layers
model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

Model compilation configuration

In [ ]:
optimizer = keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Function to sample the next character given the model’s predictions

In [ ]:
def sample(preds, temperature=1.0):
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds) / temperature
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  probas = np.random.multinomial(1, preds, 1)
  return np.argmax(probas)

FIt the model

In [ ]:
for epoch in range(1, 60):
  print('epoch', epoch)
  model.fit(x, y, batch_size=128, epochs=1)
  start_index = random.randint(0, len(text) - maxlen - 1)
  generated_text = text[start_index: start_index + maxlen]
  print('--- Generating with seed: "' + generated_text + '"')

  for temperature in [0.2, 0.5, 1.0, 1.2]:
    print('------ temperature:', temperature)
    sys.stdout.write(generated_text)
    for i in range(400):
        sampled = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(generated_text):
           sampled[0, t, char_indices[char]] = 1.
        preds = model.predict(sampled, verbose=0)[0]
        next_index = sample(preds, temperature)
        next_char = chars[next_index]
        generated_text += next_char
        generated_text = generated_text[1:]
        sys.stdout.write(next_char)
        sys.stdout.flush()

epoch 1
3130/3130 [==============================] - 310s 99ms/step - loss: 1.4002
--- Generating with seed: "oken of as a being existing from all eternity, and
with refe"
------ temperature: 0.2
oken of as a being existing from all eternity, and
with referent from the subject and conscience and such and that who was suffering and stands from that is a subjectice and conscience and that that is a man as a man is in the condition of the same moral conscience and a disting that that is a soul is a sension and that he without the subject is as a man and that is a man is a man, and that is a man and a stands from the same subjectice of the same such an------ temperature: 0.5
an and a stands from the same subjectice of the same such and in the ways without that who is a matters to the matter of the same inconceive he
would so man is the developments who are before the subject and future that the contradicts and that preperson to conflict and prefurt to the medicious problem with the fruntar

Generates text

In [ ]:
for temperature in [0.2, 0.5, 1.0, 1.2]:
  print('------ temperature:', temperature)
  sys.stdout.write(generated_text)
  for i in range(400):
    sampled = np.zeros((1, maxlen, len(chars)))
    for t, char in enumerate(generated_text):
      sampled[0, t, char_indices[char]] = 1.
    preds = model.predict(sampled, verbose=0)[0]
    next_index = sample(preds, temperature)
    next_char = chars[next_index]
    generated_text += next_char
    generated_text = generated_text[1:]
    sys.stdout.write(next_char)
    sys.stdout.flush()

------ temperature: 0.2
inligerstinies  asmose.-it take
sidgative, and only with this same the such a such a such a such a such a such a such a such a presing and the such a such a suphing and the same and in the self and in the spirit and it is a such a such a such a such a such a such a such a such a such a such a such a such a such a vilition and all the self and self the such a such a such a such a such a presing and the strom in the such a self and and in the something is a ------ temperature: 0.5
 the strom in the such a self and and in the something is a such impains and man is a such a wore in the diffical an east we indiesting in the specision and all the such the suffiring in the reason there or are of the indeasive of the prigions of the some spirit in the simper and the sirding and under and a such has with mess why his was a priction and endur of the domant and perhaps of the selfical and with the wore of the condigine the reliby and pridend------ temperature: 1.0
al and w

fit the model gpt

In [ ]:
model.fit(x, y, batch_size=128, epochs=1)

1565/1565 [==============================] - 155s 98ms/step - loss: 2.0211


In [ ]:
# Generating text
start_index = random.randint(0, len(text) - maxlen - 1)
generated_text = text[start_index: start_index + maxlen]

In [ ]:
for temperature in [0.2, 0.5, 1.0, 1.2]:
    print('\n------ Temperature:', temperature)
    sys.stdout.write(generated_text)

    for i in range(400):
        sampled = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(generated_text):
            sampled[0, t, char_indices[char]] = 1.

        preds = model.predict(sampled, verbose=0)[0]
        next_index = np.random.choice(len(chars), p=preds)
        next_char = chars[next_index]

        generated_text += next_char
        generated_text = generated_text[1:]

        sys.stdout.write(next_char)
        sys.stdout.flush()
print()


------ Temperature: 0.2
t pleasure is caused by the display of
our power over others"--to faciment un" live elstic inselverness,
incentinial they sachout the adestief farct is eacist, in an
aisted ain jidgest of to a fance and
untrass oäntipely, givical, of seladie with,
his veiss suffect as cormens oully man he causlibfe, )cioltic what is compoin such he brith nable of manj
sam-nefich of which constincts in elrut" and world zelief] vae is or edististreal
the wansdiw
lomes and is
------ Temperature: 0.5
orld zelief] vae is or edististreal
the wansdiw
lomes and is a hundiorient to have ont digars. the rades on nime," regencess of the weciss no a2ctil_tappitihments in the _emsembes? for ?outh,
yeach and atsiter veby greary cintire if still may is to the slaint
terpomally uncelta;
and becaust delight pothiom of which horo,,
-it which has, the racidust quais althelty" with incimal litere, and
issting why hand pry, there pless. of any and is mans to doing beli
------ Temperature: 1.0
 why 